In [1]:
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
spark = SparkSession.builder \
       .master("local[*]") \
       .appName("test") \
       .getOrCreate()

24/04/05 04:24:45 WARN Utils: Your hostname, ahona-ThinkPad-X270-W10DG resolves to a loopback address: 127.0.1.1; using 192.168.43.6 instead (on interface wlp3s0)
24/04/05 04:24:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/05 04:24:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.options(header='true', inferSchema='true').csv('fhv_tripdata_2021-01.csv')

In [4]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2021-01-01 00:27:00|2021-01-01 00:44:00|        null|        null|   null|                B00009|
|              B00009|2021-01-01 00:50:00|2021-01-01 01:07:00|        null|        null|   null|                B00009|
|              B00013|2021-01-01 00:01:00|2021-01-01 01:51:00|        null|        null|   null|                B00013|
|              B00037|2021-01-01 00:13:09|2021-01-01 00:21:26|        null|          72|   null|                B00037|
|              B00037|2021-01-01 00:38:31|2021-01-01 00:53:44|        null|          61|   null|                B00037|
|              B00037|2021-01-01 00:59:0

In [5]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [11]:
from pyspark.sql import types

In [12]:
schema = types.StructType([ 
   types.StructField('dispatching_base_num', types.StringType(), True), 
   types.StructField('pickup_datetime', types.TimestampType(), True), 
   types.StructField('dropoff_datetime', types.TimestampType(), True),
   types.StructField('PULocationID', types.IntegerType(), True), 
   types.StructField('DOLocationID', types.IntegerType(), True), 
   types.StructField('SR_Flag', types.StringType(), True), 
   types.StructField('Affiliated_base_number', types.StringType(), True)
 ])


In [13]:
df = spark.read.option('header','true').schema(schema).csv('fhv_tripdata_2021-01.csv')

In [14]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2021-01-01 00:27:00|2021-01-01 00:44:00|        null|        null|   null|                B00009|
|              B00009|2021-01-01 00:50:00|2021-01-01 01:07:00|        null|        null|   null|                B00009|
|              B00013|2021-01-01 00:01:00|2021-01-01 01:51:00|        null|        null|   null|                B00013|
|              B00037|2021-01-01 00:13:09|2021-01-01 00:21:26|        null|          72|   null|                B00037|
|              B00037|2021-01-01 00:38:31|2021-01-01 00:53:44|        null|          61|   null|                B00037|
|              B00037|2021-01-01 00:59:0

In [15]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [16]:
df.head(10)

[Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 27), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 44), PULocationID=None, DOLocationID=None, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 50), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 7), PULocationID=None, DOLocationID=None, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00013', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 1), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 51), PULocationID=None, DOLocationID=None, SR_Flag=None, Affiliated_base_number='B00013'),
 Row(dispatching_base_num='B00037', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 13, 9), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 21, 26), PULocationID=None, DOLocationID=72, SR_Flag=None, Affiliated_base_number='B00037'),
 Row(dispatching_base_num='B00037', pickup_datetime=datetime.datetime(202

In [18]:
df = df.repartition(24)

In [22]:
df.write.parquet('fhhv/2021/01', mode='overwrite')

In [20]:
!ls

Dockerfile		  head.csv		  test_spark.ipynb
fhhv			  ingest_data_pyspark.py  venv
fhv_tripdata_2021-01.csv  taxi_zone_lookup.csv	  yellow_data
green_data		  test_01.ipynb		  zones


In [25]:
df = spark.read.parquet('fhhv/2021/01')

In [27]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [32]:
df.select('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID') \
  .filter(df.Affiliated_base_number=='B00037') \
  .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-09 15:51:50|2021-01-09 16:16:17|        null|         188|
|2021-01-07 14:59:13|2021-01-07 15:28:14|        null|          61|
|2021-01-08 13:40:01|2021-01-08 13:54:08|        null|         188|
|2021-01-07 14:05:05|2021-01-07 14:21:32|        null|          71|
|2021-01-09 13:11:50|2021-01-09 13:18:25|        null|          85|
|2021-01-07 13:37:17|2021-01-07 14:04:47|        null|          26|
|2021-01-03 07:37:03|2021-01-03 07:41:02|        null|         188|
|2021-01-01 02:02:09|2021-01-01 02:10:01|        null|          85|
|2021-01-09 14:36:38|2021-01-09 14:41:42|        null|          72|
|2021-01-04 04:15:00|2021-01-04 04:46:02|        null|         138|
|2021-01-07 05:09:07|2021-01-07 05:28:12|        null|          76|
|2021-01-08 12:40:45|2021-01-08 12:52:24|       

In [33]:
from pyspark.sql import functions as F

In [36]:
df \
  .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
  .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
  .select('pickup_date', 'dropoff_date') \
  .show()

+-----------+------------+
|pickup_date|dropoff_date|
+-----------+------------+
| 2021-01-05|  2021-01-05|
| 2021-01-04|  2021-01-04|
| 2021-01-08|  2021-01-08|
| 2021-01-06|  2021-01-06|
| 2021-01-02|  2021-01-02|
| 2021-01-06|  2021-01-06|
| 2021-01-02|  2021-01-02|
| 2021-01-04|  2021-01-04|
| 2021-01-01|  2021-01-01|
| 2021-01-07|  2021-01-07|
| 2021-01-06|  2021-01-06|
| 2021-01-05|  2021-01-05|
| 2021-01-06|  2021-01-06|
| 2021-01-01|  2021-01-01|
| 2021-01-07|  2021-01-07|
| 2021-01-03|  2021-01-03|
| 2021-01-02|  2021-01-02|
| 2021-01-08|  2021-01-08|
| 2021-01-05|  2021-01-05|
| 2021-01-03|  2021-01-03|
+-----------+------------+
only showing top 20 rows

